# Recognize Face

In [ ]:
import face_recognition
import cv2
import numpy as np

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(1)

# # Load a sample picture and learn how to recognize it.
# abhay_image = face_recognition.load_image_file("Abhay.png")
# abhay_face_encoding = face_recognition.face_encodings(abhay_image)[0]

# # Load a second sample picture and learn how to recognize it.
# ajay_image = face_recognition.load_image_file("Ajay.png")
# ajay_face_encoding = face_recognition.face_encodings(ajay_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = (np.load('known_face_encodings.npy')).tolist()
known_face_names = (np.load('known_face_names.npy')).tolist()

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        # rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            print(matches)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]
            #     print(name)

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

            # print(face_distances[0])

            # best_match_index = np.argmin(face_distances)
            # if matches[best_match_index]:
            #     name = known_face_names[best_match_index]
                
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index] and face_distances[best_match_index] < 0.5:
                name = known_face_names[best_match_index]

            

            face_names.append(name)

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

# Show Video

In [1]:
import cv2

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

# Generate QR

In [2]:
import qrcode
import string
import random

# Generate QR codes for 14 people
for i in range(1, 15):
    ran = ''.join(random.choices(string.ascii_lowercase, k = 6))
    # Create a unique ID for each person
    unique_id = f"{ran}_{i}"
    # Generate the QR code using the unique ID
    qr = qrcode.QRCode(version=1, box_size=10, border=5)
    qr.add_data(unique_id)
    qr.make(fit=True)
    # Save the QR code as an image file
    img = qr.make_image(fill_color="black", back_color="white")
    img.save(f"{unique_id}.png")


# Scan QR

In [ ]:
import cv2
import qrcode

def get_name(qr):
    # name_email_dict = dict(zip(known_face_names, known_face_mails))
    # name_email_dict
    # create dictionary of names and emails
    qr_name_dict = {
        'wuahay_1':      'Abhay'
    }

    # return email of person whose name is given as input
    # If the name is not in the dictionary, get() returns None by default.
    return qr_name_dict.get(qr)

# Load the webcam
cap = cv2.VideoCapture(0)

# initialize the cv2 QRCode detector
detector = cv2.QRCodeDetector()

# Keep scanning until the user quits
while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # detect and decode
    decoded_qr, vertices_array, binary_qrcode = detector.detectAndDecode(frame)

    # if there is a QR code
    # print the data
    if vertices_array is not None:
        decoded_name_from_qr = get_name(decoded_qr)
        if decoded_name_from_qr is not None:
            print(f"Its {decoded_name_from_qr}")
    
    # # Find the QR codes in the frame
    # qr_codes = qrcode.decode(frame)
    
    # # Check if any of the QR codes match the unique ID of a person
    # for qr_code in qr_codes:
    #     decoded_qr = qr_code.data.decode()
    #     decoded_name_from_qr = get_name(decoded_qr)
    #     if decoded_name_from_qr != None:
    #         print(f"Its {decoded_name_from_qr}")
    #     else:
    #         continue
    
    # Display the frame
    cv2.imshow('frame', frame)
    
    # Quit if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


# Send QR in mail

In [ ]:
import random
import qrcode
from io import BytesIO
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage


def send_qr_code(recipient_email):
    # Generate random 6-digit number
    code = random.randint(100000, 999999)

    # Create QR code
    qr = qrcode.QRCode(version=1, box_size=10, border=5)
    qr.add_data(str(code))
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")

    # Convert image to bytes
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    buffer.seek(0)
    img_bytes = buffer.getvalue()

    # Set up email information
    sender_email_address = '******@outlook.com'
    sender_email_password = '*******'

    subject = 'TL Security Code'
    instruction = 'Scan this QR code to confirm your identity'
    body = f'{instruction}'

    # Set up the SMTP server
    smtp_server = 'smtp.outlook.com'
    smtp_port = 587
    smtp_connection = smtplib.SMTP(smtp_server, smtp_port)
    smtp_connection.starttls()
    smtp_connection.login(sender_email_address, sender_email_password)

    # Create the email message with attached QR code image
    message = MIMEMultipart()
    message['From'] = sender_email_address
    message['To'] = recipient_email
    message['Subject'] = subject
    message.attach(MIMEText(body))
    image = MIMEImage(img_bytes)
    message.attach(image)

    # Send the email
    smtp_connection.sendmail(sender_email_address, recipient_email, message.as_string())

    # Close the SMTP connection
    smtp_connection.quit()

    # Return code
    return code

send_qr_code('**********')